# Скрэпинг стенограмм заседаний Государственной Думы

Источник: http://transcript.duma.gov.ru/

Цель: получить обсуждение законопроектов в первом чтении для каждого комитета для каждой сессии Думы с 2011 года

## Алгоритм:

1) Вручную указать ссылки на страницы со списком хроникам для каждого созыва (страницы, на которых собраны ссылки).

   Переменная: **session_X_chronicles_pages**
   
2) Вручную указать ссылки на страницы со списком стенограмм для каждого созыва.

   Переменная: **session_X_stenograms_pages**

3) Получить ссылки на все хроники созыва X
   
   Функция: **get_all_chronicles_links**(session_chronicles_pages)   

   Переменная: **session_X_chronicles_links**
   
4) Получить ссылки на все стенограммы созыва X
   
   Функция: **get_all_stenograms_links**(session_stenograms_pages)   

   Переменная: **session_X_stenograms_links** 
   
5) Из каждой хроники получить список пунктов повестки, в которых законопроекты первого чтения имели обсуждение (были заданы вопросы). 

   Функция для одной хроники: **get_discussed_numbers**
    
   Функция для списка ссылок на хроники: **get_all_discussed_numbers**(session_chronicles_links)
    
   Переменная: **session_X_discussed_numbers**

In [15]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from transliterate import translit

In [16]:
list_of_commities = ['аграрным вопросам',
 'безопасности и противодействию коррупции',
 'бюджету и налогам',
 'вопросам семьи, женщин и детей',
 'вопросам собственности, земельным и имущественным отношениям',
 'государственному строительству и законодательству',
 'делам Содружества Независимых Государств, евразийской интеграции и связям с соотечественниками',
 'делам национальностей',
 'защите конкуренции',
 'информационной политике, информационным технологиям и связи',
 'контролю',
 'культуре',
 'малому и среднему предпринимательству',
 'международным делам',
 'молодежной политике',
 'науке и высшему образованию',
 'обороне',
 'охране здоровья',
 'промышленности и торговле',
 'просвещению',
 'развитию Дальнего Востока и Арктики',
 'развитию гражданского общества, вопросам общественных и религиозных объединений',
 'региональной политике и местному самоуправлению',
 'строительству и жилищно-коммунальному хозяйству',
 'транспорту и развитию транспортной инфраструктуры',
 'труду, социальной политике и делам ветеранов',
 'туризму и развитию туристической инфраструктуры',
 'физической культуре и спорту',
 'финансовому рынку',
 'экологии, природным ресурсам и охране окружающей среды',
 'экономической политике',
 'энергетике']

In [29]:
def get_links_chronicles(url):
    '''
    Function will return a list in which 
    every element shows
    [date of session][link to chronicle of session]
    '''
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    raw_chronicles = []
    for link in soup.find_all("a", href = re.compile(r"^/node/\d+/")):
        raw_chronicles.append(str(link))    

    links_chronicles = []
    beginning = 'http://transcript.duma.gov.ru/'
    for i in raw_chronicles:
        link_start = i.find('node')
        link = beginning +  i[link_start:20]
        date_start = i.find('Хроника')
        date_ending = i.find('<', i.find('Хроника'))
        date = i[date_start:date_ending]
        links_chronicles.append([date, link])

    return links_chronicles

In [17]:
def get_links_stenograms(url):
    
    '''    
    На вход: ссылка на страницу со ссылками на стенограмму.
    
    Возвращает: список формата    
    [дата сессии][ссылка на стенограмму]
    
    '''
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    raw_stenograms = []
    for link in soup.find_all("a", href = re.compile(r"^/node/\d+/")):
        raw_stenograms.append(str(link))    

    links_stenograms = []
    beginning = 'http://transcript.duma.gov.ru/'
    for i in raw_stenograms:
        link_start = i.find('node')
        link = beginning +  i[link_start:20]
        date_start = i.find('Стенограмма')
        date_ending = i.find('<', i.find('Стенограмма'))
        date = i[date_start:date_ending]
        links_stenograms.append([date, link])

    return links_stenograms

In [31]:
def get_all_chronicles_links(session_chronicles_pages):    
    
    '''
    На вход: страницы, на которых много ссылкок на хроники.
    Возвращает: список всех ссылок на все эти хроники. 
    '''
    
    session_chronicles_links = []
    for i in session_chronicles_pages:
        chron = get_links_chronicles(i)
        only_links = list(map(lambda x: x[1], chron))
        session_chronicles_links += (only_links)
        
    return session_chronicles_links

In [34]:
def get_all_stenograms_links(session_stenograms_pages):
    
    '''
    На вход: страницы, на которых много ссылкок на стенограммы.
    Возвращает: словарь всех ссылок на все эти стенограммы
    (ключ - дата)
    '''
    
    session_stenograms_links = []

    for i in session_stenograms_pages:
        session_stenograms_links.extend(get_links_stenograms(i))
        
    
    # создадим словарь, чтобы было удобнее обращаться к элементам

    session_stenograms_links_dict = dict()
    for i in session_stenograms_links:
        key = i[0]
        key = key[key.find(' ', key.find('засед')) +1 :]
        value = i[1]
        session_stenograms_links_dict[key] = value
        
    return session_stenograms_links_dict

In [40]:
def get_discussed_numbers(url):
    
    '''
    на вход - ссылка на хронику.
    returns:
    * дата заседания
    * пункт повестки, в котором было обсуждение,
    * индикатор того, был ли содоклад,
    * какой комитет выступал    
    
    '''

    #url = 'http://transcript.duma.gov.ru/node/5969/'

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    date_start = soup.text.find('Хроника заседания')
    date_end = soup.text.find('.')
    date = soup.text[date_start: date_end+1]    

    start_phrase = 'депутаты приступили к рассмотрению законопроектов в первом чтении'
    if start_phrase is None:
        return None
    start_tag = soup.find(lambda tag: tag.name == 'p' and start_phrase in tag.text)
    if start_tag is None:
        return None
    start_index = soup.find_all('p').index(start_tag)

    results = []

    for p in soup.find_all('p')[start_index+1:]:
        phrases = ['ответил на вопросы', 'ответила на вопросы', \
                   'ответили на вопросы', 'на вопросы ответил', \
                   'на вопросы ответила', 'на вопросы ответили']
        if any(phrase in p.text.lower() for phrase in phrases):
            item = []
            item.append(date)
            paragraph = p.text.lower()
            start = paragraph.find('повестки дня')
            number = paragraph[start-3: start]
            item.append(number)
            
            other_speaker = 0
            if 'содоклад' in paragraph:
                other_speaker = 1
            if 'содоклад' not in paragraph:
                if 'выступил' in paragraph and 'изложил' in paragraph:
                    other_speaker = 1
                if 'выступил' in paragraph and paragraph.count('выступил') > 1:
                    other_speaker = 1
                    
            item.append(other_speaker)
            if 'комитета государственной думы по' in paragraph:
                found_options = ['']
                # устранение ошибки по типу: "культуре", "культуре и спорту"
                # возьмем самое длинное (=самое полное) совпадение
                for c in list_of_commities:
                    if c in paragraph: 
                        found_options.append(c)
                item.append(max(found_options))
                        
            else:
                item.append(0)
            results.append(item)
            
    results = pd.DataFrame(results)
    results.columns = ['дата', 'пункт повестки', 'наличие содоклада', 'комитет']
            
    return results

In [38]:
def get_all_discussed_numbers(session_chronicles_links):
    
    '''
    На вход: ссылки на все хроники
    На выход: список указаний на то, в какую дату какой пункт повестки
    имел обсуждение (были заданы вопросы), был ли представлен содоклад,
    к какому комитету относится обсуждение
    
    '''
    
    session_discussed_numbers = []
    
    for i in session_chronicles_links:
        session_discussed_numbers.append(get_discussed_numbers(i))
        
    session_discussed_numbers = [i for i in session_discussed_numbers if i is not None]
    
    session_discussed_numbers = pd.concat(session_discussed_numbers)
        
    return session_discussed_numbers    

In [22]:
# словарь с соответствие названия коммитета и транслита первых пяти букв

dict_translits = dict()
for i in list_of_commities:
    translited = translit(i[:5], 'ru', reversed=True)
    translited = translited.replace("'", '')
    dict_translits[i] = translited
dict_translits

{'аграрным вопросам': 'agrar',
 'безопасности и противодействию коррупции': 'bezop',
 'бюджету и налогам': 'bjudzhe',
 'вопросам семьи, женщин и детей': 'vopro',
 'вопросам собственности, земельным и имущественным отношениям': 'vopro',
 'государственному строительству и законодательству': 'gosud',
 'делам Содружества Независимых Государств, евразийской интеграции и связям с соотечественниками': 'delam',
 'делам национальностей': 'delam',
 'защите конкуренции': 'zaschit',
 'информационной политике, информационным технологиям и связи': 'infor',
 'контролю': 'kontr',
 'культуре': 'kult',
 'малому и среднему предпринимательству': 'malom',
 'международным делам': 'mezhdu',
 'молодежной политике': 'molod',
 'науке и высшему образованию': 'nauke',
 'обороне': 'oboro',
 'охране здоровья': 'ohran',
 'промышленности и торговле': 'promy',
 'просвещению': 'prosv',
 'развитию Дальнего Востока и Арктики': 'razvi',
 'развитию гражданского общества, вопросам общественных и религиозных объединений': 'r

In [43]:
def get_variables_links_by_commities(session_discussed_numbers, n):
    
    '''
    Функция создает переменные со всеми датами обсужденных вопросов
    для каждого комитета отдельно
    
    Возвращает список из созданных переменных
    '''    
    session_dialogies_by_commities = []
    
    
    n = str(n)
    for i in set(session_discussed_numbers['комитет']):
        if i not in [0, None]:
            translited = translit(i[:5], 'ru', reversed=True)
            name = 'session_' + n + '_dialog_commitee_'+ str(translited)
            name = name.replace("'", '')
            globals()[name] = session_discussed_numbers[session_discussed_numbers['комитет'] == i]
            session_dialogies_by_commities.append(name)
        if i == None:
            name = 'session_' + n + '_dialog_commitee_None'
            globals()[name] = session_discussed_numbers[session_discussed_numbers['комитет'] == i]
            session_dialogies_by_commities.append(name)
        if i == 0:
            name = 'session_' + n + '_dialog_commitee_0'
            globals()[name] = session_discussed_numbers[session_discussed_numbers['комитет'] == i]
            session_dialogies_by_commities.append(name)        
             
    return session_dialogies_by_commities

In [24]:
def get_clean_transcript(url):
    
    '''
    На вход: ссылка на стенограмму
    На выход: транскрипт стенограммы в формате 
    [кто говорит, что говорит]
    
    '''
    
    #url = "http://transcript.duma.gov.ru/node/5750/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    soup.i.decompose()

    discussion = []

    who = ''
    what = ''
    for i in soup.find_all(['p']):
        if i.find(['b']):
            if len(who) > 0:
                discussion.append([who, what])
            who = (i.find(['b']).text).replace(',', '')
            what = (i.text).replace(i.find(['b']).text, '')
            if 'фракция' in what or 'официальный представитель' in what:
                what = ''
        else:
            what += i.text

    return discussion

In [62]:
def get_certain_discussion(url, number, additional_speech):
    
    '''
    Функция возвращает сессию вопросов и ответов по определенному пункту повестки дня
    На вход: ссылка, номер повестки, индикатор наличия содоклада
    На выход: 
        список: за вопросом следует ответ
    
    '''
    number = str(number)
    #url = "http://transcript.duma.gov.ru/node/5750/"
    corpus = get_clean_transcript(url)
    
    markers = [number + '-й вопрос',
    'пункт ' + number,
    number + '-й пункт',
    'рассмотрению вопроса ' + number,
    number + '-й, проект',
    number + '-го вопроса',
    number + ' вопроса',
    number + '-му вопросу',
    'вопрос ' + number,
    number + ' вопрос',
    'пункту ' + number ]
    
    start = 0
    
    # найти начало обсуждения определенного пункта
    for index, value in enumerate(corpus):
        if any(marker in (value[1]).lower() for marker in markers)\
            and value[0] == 'Председательствующий.': # только он объявляет повестку
            start = index
            break
    
    # найти конец обсуждения
    for i in range(start, len(corpus)):
        if 'желающие выступить' in corpus[i][1].lower():
            end = i
            break
        if "результаты голосования" in corpus[i][1].lower():
            end = i
            break
    
    # вырезать диалог без дополнительных комментариев
    dialog = corpus[start:end]
    dialog[:] = [i for i in dialog if i[0] != 'Председательствующий.']
    dialog[:] = [i for i in dialog if i[0] != 'Из зала.']
    
    # склеить реплики одного и того же человека (могли быть прерваны председателем)
    for i in range(1, len(dialog)):
        if dialog[i][0] == dialog[i-1][0]:
            name  = dialog[i][0]
            content = dialog[i-1][1] + dialog[i][1]
            dialog[i-1] = [] # заменим пустым списком, чтобы не изменять длину диалога
            dialog[i] = [name, content]
            
    # убрать пустые элементы
    dialog[:] = [i for i in dialog if len(i) > 0]
    
    # fining start
    print(dialog)
    return 
    #fixing finish
    
    person1 = dialog[0][0]
    if additional_speech == 1:
        person2 = dialog[1][0] 
    
    # убрать выступление докладчика и содокладчика, интересуют только вопросы и ответы
    
    dialog[:] = dialog[1:]
    if additional_speech == 1:   
        dialog[:] = dialog[1:]
        
        speakers = [person1, person2]
        
        # склеить ответы докладчика и содокладчика, если они рядом 
        # (отвечают на один и тот же вопрос)
        
        for i in range(1, len(dialog)):

            if dialog[i][0] in speakers and dialog[i-1][0] in speakers: 
                name  = 'оба спикера'
                content = dialog[i-1][1] + dialog[i][1]
                dialog[i-1] = [] # заменим пустым списком, чтобы не изменять длину диалога
                dialog[i] = [name, content]
            
    # убрать пустые элементы
    dialog[:] = [i for i in dialog if len(i) > 0]       
    
    # оставить только реплики без указания на то, кто говорит
    dialog[:] = [i[1] for i in dialog]
    
    return dialog

In [26]:
def get_all_clean_dialogues_by_commitee(session_dialog_commitee, session_stenograms_links_dict):
    
    '''
    На вход: переменная со списком, указывающим на даты заседения и
    для имеющих обсуждение пунктов повестки
    
    На выход: список с последовательными сессиями вопросов и ответов
    для определенного коммитета
    '''
    session_all_dialogs = []

    for row_chron in session_dialog_commitee.values.tolist():
        chron_date = row_chron[0]
        chron_date = chron_date[chron_date.find(' ', chron_date.find('заседания')) +1:]
        question_number = row_chron[1]
        indication = row_chron[2]
    
        link_stenogram = session_stenograms_links_dict[chron_date]
        #print(link_stenogram, int(question_number), indication)
        session_all_dialogs.extend(
        get_certain_discussion(link_stenogram, int(question_number), indication))
    
    return session_all_dialogs

# Сбор данных

In [ ]:
# session_8_chronicles_pages = []
# session_8_stenograms_pages = []

In [27]:
session_8_stenograms_pages = [
'http://transcript.duma.gov.ru/search/?sessid=5687&doctype=3&dt_start=&dt_end=&phrase1=',\
'http://transcript.duma.gov.ru/search/?by=date&sessid=5767&doctype=3&dt_start=&dt_end=&phrase1=&PAGEN_1=2',\
'http://transcript.duma.gov.ru/search/?sessid=5767&doctype=3&dt_start=&dt_end=&phrase1=',\
'http://transcript.duma.gov.ru/search/?sessid=5888&doctype=3&dt_start=&dt_end=&phrase1=',\
'http://transcript.duma.gov.ru/search/?sessid=5983&doctype=3&dt_start=&dt_end=&phrase1=']

# Ссылки на страницы с хрониками четырех сессий 8 созыва:

session_8_chronicles_pages = \
['http://transcript.duma.gov.ru/search/?sessid=5687&doctype=2&dt_start=&dt_end=&phrase1=',\
'http://transcript.duma.gov.ru/search/?sessid=5767&doctype=2&dt_start=&dt_end=&phrase1=',\
'http://transcript.duma.gov.ru/search/?sessid=5888&doctype=2&dt_start=&dt_end=&phrase1=',\
'http://transcript.duma.gov.ru/search/?sessid=5983&doctype=2&dt_start=&dt_end=&phrase1=']

In [32]:
session_8_chronicles_links = get_all_chronicles_links(session_8_chronicles_pages)

In [35]:
session_8_stenograms_links_dict = get_all_stenograms_links(session_8_stenograms_pages)

In [41]:
session_8_discussed_numbers = get_all_discussed_numbers(session_8_chronicles_links)

In [44]:
session_8_dialogies_by_commities = \
    get_variables_links_by_commities(session_8_discussed_numbers, 8)

session_8_dialogies_by_commities

['session_8_dialog_commitee_0',
 'session_8_dialog_commitee_',
 'session_8_dialog_commitee_regio',
 'session_8_dialog_commitee_nauke',
 'session_8_dialog_commitee_ekolo',
 'session_8_dialog_commitee_agrar',
 'session_8_dialog_commitee_stroi',
 'session_8_dialog_commitee_vopro',
 'session_8_dialog_commitee_razvi',
 'session_8_dialog_commitee_trudu',
 'session_8_dialog_commitee_gosud',
 'session_8_dialog_commitee_finan',
 'session_8_dialog_commitee_oboro',
 'session_8_dialog_commitee_mezhdu',
 'session_8_dialog_commitee_kontr',
 'session_8_dialog_commitee_trans',
 'session_8_dialog_commitee_bjudzhe',
 'session_8_dialog_commitee_vopro',
 'session_8_dialog_commitee_ekono',
 'session_8_dialog_commitee_promy',
 'session_8_dialog_commitee_infor',
 'session_8_dialog_commitee_ohran',
 'session_8_dialog_commitee_prosv',
 'session_8_dialog_commitee_fizich',
 'session_8_dialog_commitee_bezop',
 'session_8_dialog_commitee_delam',
 'session_8_dialog_commitee_energ',
 'session_8_dialog_commitee_zasch

In [45]:
session_8_all_dialogs_culture = get_all_clean_dialogues_by_commitee(\
       session_8_dialog_commitee_kult, session_8_stenograms_links_dict)

In [47]:
len(session_8_all_dialogs_culture)

78

### Повторим пример для другого коммитета

In [60]:
session_8_dialog_commitee_nauke

,дата,пункт повестки,наличие содоклада,комитет
2,Хроника заседания 15 декабря 2021 г.,13,1,науке и высшему образованию
3,Хроника заседания 14 декабря 2021 г.,30,1,науке и высшему образованию
5,Хроника заседания 09 ноября 2021 г.,19,1,науке и высшему образованию
1,Хроника заседания 08 декабря 2022 г.,26,1,науке и высшему образованию
2,Хроника заседания 08 декабря 2022 г.,27,0,науке и высшему образованию
0,Хроника заседания 08 ноября 2022 г.,27,0,науке и высшему образованию
10,Хроника заседания 04 октября 2022 г.,.2,1,науке и высшему образованию
1,Хроника заседания 13 сентября 2022 г.,6,1,науке и высшему образованию
3,Хроника заседания 20 апреля 2023 г.,17,1,науке и высшему образованию
0,Хроника заседания 16 марта 2023 г.,12,1,науке и высшему образованию


In [64]:
session_8_all_dialogs_nauke = get_all_clean_dialogues_by_commitee(\
       session_8_dialog_commitee_nauke, session_8_stenograms_links_dict)

[]


TypeError: 'NoneType' object is not iterable

In [ ]:
len(session_8_all_dialogs_)

## Повторим для 7 созыва

In [63]:
session_7_chronicles_pages = \
['http://transcript.duma.gov.ru/search/?sessid=4519&doctype=2&dt_start=&dt_end=&phrase1=',
'http://transcript.duma.gov.ru/search/?sessid=4573&doctype=2&dt_start=&dt_end=&phrase1=',
'http://transcript.duma.gov.ru/search/?by=date&sessid=4573&doctype=2&dt_start=&dt_end=&phrase1=&PAGEN_1=2',
 'http://transcript.duma.gov.ru/search/?sessid=4716&doctype=2&dt_start=&dt_end=&phrase1=',
 'http://transcript.duma.gov.ru/search/?sessid=4806&doctype=2&dt_start=&dt_end=&phrase1=',
 'http://transcript.duma.gov.ru/search/?by=date&sessid=4806&doctype=2&dt_start=&dt_end=&phrase1=&PAGEN_1=2',
 'http://transcript.duma.gov.ru/search/?sessid=4961&doctype=2&dt_start=&dt_end=&phrase1=',
 'http://transcript.duma.gov.ru/search/?by=date&sessid=4961&doctype=2&dt_start=&dt_end=&phrase1=&PAGEN_1=2',
 'http://transcript.duma.gov.ru/search/?sessid=5081&doctype=2&dt_start=&dt_end=&phrase1=',
 'http://transcript.duma.gov.ru/search/?by=date&sessid=5081&doctype=2&dt_start=&dt_end=&phrase1=&PAGEN_1=2',
 'http://transcript.duma.gov.ru/search/?sessid=5256&doctype=2&dt_start=&dt_end=&phrase1=',
 'http://transcript.duma.gov.ru/search/?by=date&sessid=5256&doctype=2&dt_start=&dt_end=&phrase1=&PAGEN_1=2',
 'http://transcript.duma.gov.ru/search/?sessid=5368&doctype=2&dt_start=&dt_end=&phrase1=',
 'http://transcript.duma.gov.ru/search/?by=date&sessid=5368&doctype=2&dt_start=&dt_end=&phrase1=&PAGEN_1=2',
 'http://transcript.duma.gov.ru/search/?sessid=5503&doctype=2&dt_start=&dt_end=&phrase1=',
 'http://transcript.duma.gov.ru/search/?sessid=5579&doctype=2&dt_start=&dt_end=&phrase1=',
 'http://transcript.duma.gov.ru/search/?by=date&sessid=5579&doctype=2&dt_start=&dt_end=&phrase1=&PAGEN_1=2'



]
session_7_stenograms_pages = []